In [24]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, regularizers

# Load train and test datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Checking information of the train data
train_df.info()

# Initialize LabelEncoder and dictionary to store encoders for each column
label_encoders = {}

# List of categorical columns to encode
categorical_columns = ['model', 'motor_type', 'wheel', 'color', 'status', 'type']

# Apply LabelEncoder to each categorical column for both train and test
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    train_df[col] = label_encoders[col].fit_transform(train_df[col])
    test_df[col] = test_df[col].apply(lambda x: label_encoders[col].transform([x])[0] if x in label_encoders[col].classes_ else -1)

# Function to convert running values from 'km' to 'miles'
def convert_running(value):
    if 'km' in value:
        kilometers = int(value.split()[0])
        miles = kilometers * 0.621371
        return f"{miles:.2f} miles"
    elif 'miles' in value:
        return value
    else:
        return value 

# Apply the conversion for both train and test datasets
train_df['running'] = train_df['running'].apply(convert_running)
test_df['running'] = test_df['running'].apply(convert_running)

# Clean and convert 'running' column to numeric values
train_df['running'] = train_df['running'].str.replace('miles', '').str.strip()
test_df['running'] = test_df['running'].str.replace('miles', '').str.strip()
train_df['running'] = pd.to_numeric(train_df['running'], errors='coerce').fillna(0).astype('int64')
test_df['running'] = pd.to_numeric(test_df['running'], errors='coerce').fillna(0).astype('int64')

# Drop duplicates from the training data
train_df = train_df.drop_duplicates()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   object 
 5   color         1642 non-null   object 
 6   type          1642 non-null   object 
 7   status        1642 non-null   object 
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 128.4+ KB


In [26]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1633 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1633 non-null   int32  
 1   year          1633 non-null   int64  
 2   motor_type    1633 non-null   int32  
 3   running       1633 non-null   int64  
 4   wheel         1633 non-null   int32  
 5   color         1633 non-null   int32  
 6   type          1633 non-null   int32  
 7   status        1633 non-null   int32  
 8   motor_volume  1633 non-null   float64
 9   price         1633 non-null   int64  
dtypes: float64(1), int32(6), int64(3)
memory usage: 102.1 KB


In [30]:

# Prepare features and target variable
X = train_df.drop(['price', 'wheel'], axis = 1) # Replace 'target_column' with your actual target column name
y = train_df['price']  # Your target variable

# Split the data into training and validation sets
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
import tensorflow as tf
print(tf.__version__)

2.19.0-dev20241007


In [9]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install scikeras[tensorflow-cpu]

Note: you may need to restart the kernel to use updated packages.


In [34]:
from scikeras.wrappers import KerasRegressor

In [35]:

# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [19]:
X_train.shape, X_temp.shape, y_train.shape, y_temp.shape

NameError: name 'X_train' is not defined

In [129]:
X_test, X_val, y_test, y_val = train_test_split( X_temp, X_temp, test_size=0.5, random_state=42)

In [131]:
X_test.shape, X_val.shape, y_test.shape, y_val.shape

((163, 8), (164, 8), (163, 8), (164, 8))

In [133]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_temp_scaled = scaler.transform(X_temp)
X_val_scaled = scaler.transform(X_val)
X_scaled = scaler.transform(test_df.drop(columns=['Id', 'wheel']))  # Adjust as necessary

In [36]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_scaled = scaler.transform(test_df.drop(columns=['Id', 'wheel']))

In [38]:
import warnings
warnings.filterwarnings('ignore')

In [53]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
from keras import layers, models
 
def create_model(learning_rate=0.01):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(X.shape[1],)))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model
 
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=10, learning_rate=0.001, verbose=0)
# `KerasRegressor(learning_rate=0.001)
param_grid = {'learning_rate': [0.001, 0.01, 0.1]}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)
 
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')

Best: -0.2470929225285848 using {'learning_rate': 0.001}


In [25]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])

AttributeError: 'KerasRegressor' object has no attribute 'compile'

In [40]:
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasRegressor
from keras import layers, models
 
def create_model(learning_rate=0.01):
    model = keras.Sequential([
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X.shape[1],)),
    # layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    # layers.Dropout(0.3),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)])  # Output layer for regression
    return model
 
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=10, learning_rate=0.001, verbose=0)
# `KerasRegressor(learning_rate=0.001)
# param_grid = {'batch_size': [10, 20, 40],
#     'epochs': [50, 100],
#     'learning_rate': [0.001, 0.01, 0.1]}
grid = RandomizedSearchCV(estimator=model, param_distributions= {'batch_size': [10, 20, 40],
    'epochs': [50, 100],
    'learning_rate': [0.001, 0.01, 0.1]}, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)
 
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')

ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 770, in fit
    self._fit(
  File "C:\Users\admin\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 928, in _fit
    self._ensure_compiled_model()
  File "C:\Users\admin\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 446, in _ensure_compiled_model
    raise ValueError("You must provide a loss or a compiled model")
ValueError: You must provide a loss or a compiled model


In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

# Function to create the model
def create_model(learning_rate=0.01):
    model = Sequential()
    model.add(Dense(30, activation='relu', input_shape=(X.shape[1],)))  # Ensure correct input shape
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))  # Output layer
    model.compile(optimizer='adam', loss='mae')  # Ensure model is compiled with loss
    return model

# Create KerasRegressor
model = KerasRegressor(build_fn=create_model, learning_rate=0.01, verbose=0)

# Define the parameter distributions
param_distributions = {
    'batch_size': [10, 20, 40],
    'epochs': [50, 100],
    'learning_rate': [0.001, 0.01, 0.1]
}

# Create the RandomizedSearchCV object
grid = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_jobs=-1, cv=3, n_iter=10)
grid_result = grid.fit(X, y)

# Print the best parameters and score
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')


AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'
Best: 0.46051810185114544 using {'learning_rate': 0.1, 'epochs': 100, 'batch_size': 10}


In [25]:
import warnings

warnings.filterwarnings('ignore')

In [44]:
model = models.Sequential([
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X.shape[1],)),
    # layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    # layers.Dropout(0.3),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)  # Output layer for regression
])

In [27]:

# Define the deep learning model with improved architecture
model = keras.Sequential([
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X.shape[1],)),
    # layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    # layers.Dropout(0.3),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)  # Output layer for regression
])


In [50]:

# Compile the model with a learning rate scheduler
optimizer = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


In [59]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])

In [49]:

# # Fit the model with early stopping
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_mae', patience=10, restore_best_weights=True)


In [52]:
model.fit(X, y,  epochs = 100, batch_size = 10)

Epoch 1/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 7141.4487 - mae: 7110.1094
Epoch 2/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3676.5120 - mae: 3641.1499
Epoch 3/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3439.1973 - mae: 3410.5674
Epoch 4/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3899.3967 - mae: 3872.0945
Epoch 5/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3302.7136 - mae: 3268.4209
Epoch 6/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3180.6609 - mae: 3144.1663
Epoch 7/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3308.7539 - mae: 3272.4934
Epoch 8/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3041.7751 - mae: 3001.5750
Epoch 9/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2808.8743 - mae: 2766.9473
Epoch 10/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3261.8987 - mae: 3212.8621
Epoch 11/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3139.4219 - mae: 3091.4375
Epoch 12/100
164/164 ━━━━━━━━━

In [53]:
model.fit(X, y,
                    epochs=200, batch_size=32, verbose=1)

Epoch 1/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2125.8291 - mae: 2020.7717
Epoch 2/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2191.0896 - mae: 2086.4016
Epoch 3/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2019.6418 - mae: 1915.8632
Epoch 4/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2025.0042 - mae: 1920.1741
Epoch 5/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2014.3474 - mae: 1912.3174
Epoch 6/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2057.5234 - mae: 1955.9714
Epoch 7/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2130.3811 - mae: 2029.1367
Epoch 8/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1926.8857 - mae: 1824.8081
Epoch 9/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2070.9814 - mae: 1970.1550
Epoch 10/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1983.7960 - mae: 1882.7870  
Epoch 11/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1936.8271 - mae: 1835.7793
Epoch 12/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/ste

In [143]:

model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), 
                    epochs=200, batch_size=32, verbose=1)

# callbacks=[early_stopping]

Epoch 1/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 15909.5254 - mae: 15906.9775 - val_loss: 9574.1611 - val_mae: 9571.6182
Epoch 2/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 15959.2031 - mae: 15956.2773 - val_loss: 9860.4951 - val_mae: 9855.4814
Epoch 3/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 15102.1104 - mae: 15095.7373 - val_loss: 11736.2207 - val_mae: 11724.7109
Epoch 4/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11280.0928 - mae: 11266.3828 - val_loss: 18136.1836 - val_mae: 18114.5742
Epoch 5/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4507.8594 - mae: 4484.5669 - val_loss: 20060.3516 - val_mae: 20036.4375
Epoch 6/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3637.7019 - mae: 3613.7273 - val_loss: 19988.3887 - val_mae: 19964.5527
Epoch 7/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3312.1526 - mae: 3288.0740 - val_loss: 20375.8828 - val_mae: 20351.5742
Epoch 8/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3296.3596 - mae: 32

In [55]:
# Evaluate the model on the validation set
val_loss, val_mae = model.evaluate(X, y)
print(f'Validation Mean Absolute Error: {val_mae:.2f}')

52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - loss: 1915.9503 - mae: 1830.9730
Validation Mean Absolute Error: 1756.84


In [64]:
# Evaluate the model on the validation set
val_loss, val_mae = model.evaluate(X_val_scaled, y_val)
print(f'Validation Mean Absolute Error: {val_mae:.2f}')

NameError: name 'X_val_scaled' is not defined

In [58]:

# Make predictions on the test set
y_predict = model.predict(X_scaled)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [62]:
# Create a DataFrame for the predictions
df_predictions = pd.DataFrame(y_predict, columns=['price'])

# Display predictions
print(df_predictions)



            price
0    18022.539062
1    16601.974609
2    20795.361328
3    14545.572266
4     5214.127441
..            ...
406  22845.863281
407  12848.049805
408  13577.025391
409  16143.666992
410  12827.865234

[411 rows x 1 columns]


In [66]:
combined_df = pd.concat([test_df['Id'], df_predictions], axis=1)

combined_df

,Id,price
0,0,18022.539062
1,1,16601.974609
2,2,20795.361328
3,3,14545.572266
4,4,5214.127441
...,...,...
406,406,22845.863281
407,407,12848.049805
408,408,13577.025391
409,409,16143.666992


In [68]:
combined_df.to_csv('fathi.csv', index=False)

In [70]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, regularizers

In [72]:
# Load train and test datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [74]:
# Checking information of the train data
train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   object 
 5   color         1642 non-null   object 
 6   type          1642 non-null   object 
 7   status        1642 non-null   object 
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 128.4+ KB


In [76]:

# Initialize LabelEncoder and dictionary to store encoders for each column
label_encoders = {}

# List of categorical columns to encode
categorical_columns = ['model', 'motor_type', 'wheel', 'color', 'status', 'type']

# Apply LabelEncoder to each categorical column for both train and test
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    train_df[col] = label_encoders[col].fit_transform(train_df[col])
    test_df[col] = test_df[col].apply(lambda x: label_encoders[col].transform([x])[0] if x in label_encoders[col].classes_ else -1)

# Function to convert running values from 'km' to 'miles'
def convert_running(value):
    if 'km' in value:
        kilometers = int(value.split()[0])
        miles = kilometers * 0.621371
        return f"{miles:.2f} miles"
    elif 'miles' in value:
        return value
    else:
        return value 

# Apply the conversion for both train and test datasets
train_df['running'] = train_df['running'].apply(convert_running)
test_df['running'] = test_df['running'].apply(convert_running)

# Clean and convert 'running' column to numeric values
train_df['running'] = train_df['running'].str.replace('miles', '').str.strip()
test_df['running'] = test_df['running'].str.replace('miles', '').str.strip()
train_df['running'] = pd.to_numeric(train_df['running'], errors='coerce').fillna(0).astype('int64')
test_df['running'] = pd.to_numeric(test_df['running'], errors='coerce').fillna(0).astype('int64')

# Drop duplicates from the training data
train_df = train_df.drop_duplicates()


In [78]:
# Prepare features and target variable
X = train_df.drop(['price', 'wheel'], axis = 1) # Replace 'target_column' with your actual target column name
y = train_df['price']  # Your target variable

In [80]:
from scikeras.wrappers import KerasRegressor

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_scaled = scaler.transform(test_df.drop(columns=['Id', 'wheel']))

import warnings
warnings.filterwarnings('ignore')

In [82]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

# Function to create the model
def create_model(learning_rate=0.01):
    model = Sequential()
    model.add(Dense(30, activation='relu', input_shape=(X.shape[1],)))  # Ensure correct input shape
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))  # Output layer
    model.compile(optimizer='adam', loss='mae')  # Ensure model is compiled with loss
    return model

# Create KerasRegressor
model = KerasRegressor(build_fn=create_model, learning_rate=0.01, verbose=0)

# Define the parameter distributions
param_distributions = {
    'batch_size': [10, 20, 40],
    'epochs': [50, 100],
    'learning_rate': [0.001, 0.01, 0.1]
}

# Create the RandomizedSearchCV object
grid = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_jobs=-1, cv=3, n_iter=10)
grid_result = grid.fit(X, y)

# Print the best parameters and score
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')



Best: 0.4636138280232747 using {'learning_rate': 0.001, 'epochs': 100, 'batch_size': 10}


In [84]:

model = models.Sequential([
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X.shape[1],)),
    # layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    # layers.Dropout(0.3),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)  # Output layer for regression
])



In [86]:


# Compile the model with a learning rate scheduler
optimizer = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])

In [88]:

model.fit(X, y,  epochs = 100, batch_size = 10)

Epoch 1/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 6612.9302 - mae: 6575.7373
Epoch 2/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4429.6460 - mae: 4390.0903
Epoch 3/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3642.0442 - mae: 3612.7854
Epoch 4/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3516.5947 - mae: 3486.6343
Epoch 5/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3358.0071 - mae: 3329.3059
Epoch 6/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3169.4722 - mae: 3138.5198
Epoch 7/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3620.1799 - mae: 3588.4934
Epoch 8/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3255.1399 - mae: 3223.3555
Epoch 9/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3558.0894 - mae: 3521.3789
Epoch 10/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3213.4409 - mae: 3171.2502
Epoch 11/100
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3178.4482 - mae: 3134.4995
Epoch 12/100
164/164 ━━━━━━━━━

In [90]:
# Evaluate the model on the validation set
val_loss, val_mae = model.evaluate(X, y)
print(f'Validation Mean Absolute Error: {val_mae:.2f}')


52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2052.6316 - mae: 1962.9252
Validation Mean Absolute Error: 1910.01


In [92]:


# Make predictions on the test set
y_predict = model.predict(X_scaled)



# Create a DataFrame for the predictions
df_predictions = pd.DataFrame(y_predict, columns=['price'])

# Display predictions
print(df_predictions)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
            price
0    18166.843750
1    15966.933594
2    21361.146484
3    14657.882812
4     4881.892090
..            ...
406  21553.349609
407  13819.800781
408  14161.121094
409  16905.031250
410  13874.337891

[411 rows x 1 columns]


In [94]:
combined_df = pd.concat([test_df['Id'], df_predictions], axis=1)

combined_df

,Id,price
0,0,18166.843750
1,1,15966.933594
2,2,21361.146484
3,3,14657.882812
4,4,4881.892090
...,...,...
406,406,21553.349609
407,407,13819.800781
408,408,14161.121094
409,409,16905.031250


In [96]:

combined_df.to_csv('fass.csv', index=False)

In [188]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, regularizers

# Load train and test datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Checking information of the train data
train_df.info()

# Initialize LabelEncoder and dictionary to store encoders for each column
label_encoders = {}

# List of categorical columns to encode
categorical_columns = ['model', 'motor_type', 'wheel', 'color', 'status', 'type']

# Apply LabelEncoder to each categorical column for both train and test
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    train_df[col] = label_encoders[col].fit_transform(train_df[col])
    test_df[col] = test_df[col].apply(lambda x: label_encoders[col].transform([x])[0] if x in label_encoders[col].classes_ else -1)

# Function to convert running values from 'km' to 'miles'
def convert_running(value):
    if 'km' in value:
        kilometers = int(value.split()[0])
        miles = kilometers * 0.621371
        return f"{miles:.2f} miles"
    elif 'miles' in value:
        return value
    else:
        return value 

# Apply the conversion for both train and test datasets
train_df['running'] = train_df['running'].apply(convert_running)
test_df['running'] = test_df['running'].apply(convert_running)

# Clean and convert 'running' column to numeric values
train_df['running'] = train_df['running'].str.replace('miles', '').str.strip()
test_df['running'] = test_df['running'].str.replace('miles', '').str.strip()
train_df['running'] = pd.to_numeric(train_df['running'], errors='coerce').fillna(0).astype('int64')
test_df['running'] = pd.to_numeric(test_df['running'], errors='coerce').fillna(0).astype('int64')

# Drop duplicates from the training data
train_df = train_df.drop_duplicates()

# Prepare features and target variable
X = train_df.drop(['price', 'wheel'], axis = 1) # Replace 'target_column' with your actual target column name
y = train_df['price']  # Your target variable


from scikeras.wrappers import KerasRegressor

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_scaled = scaler.transform(test_df.drop(columns=['Id', 'wheel']))

import warnings
warnings.filterwarnings('ignore')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   object 
 5   color         1642 non-null   object 
 6   type          1642 non-null   object 
 7   status        1642 non-null   object 
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 128.4+ KB


In [190]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras import regularizers, optimizers, callbacks

# Function to create the model
def create_model(learning_rate=0.01):
    model = Sequential()
    model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X.shape[1],)))
    model.add(BatchNormalization())  # Add Batch Normalization
    model.add(Dropout(0.3))  # Add Dropout to prevent overfitting
    model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(1))  # Output layer for regression
    optimizer = optimizers.Adam(learning_rate=learning_rate)  # Use Adam optimizer with learning rate as argument
    model.compile(optimizer=optimizer, loss='mae')  # Compile with Mean Absolute Error
    return model

# Create KerasRegressor and set the learning_rate through constructor argument
model = KerasRegressor(build_fn=create_model, verbose=0)

# Define the parameter distributions (excluding learning_rate here)
param_distributions = {
    'batch_size': [16, 32, 64],  # Increase the batch size for smoother updates
    'epochs': [50, 100, 150]
}

# Callbacks: Early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Create the RandomizedSearchCV object (learning_rate is set within create_model)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_jobs=-1, cv=3, n_iter=10)

# Fit the model with validation set
grid_result = grid.fit(X, y, callbacks=[early_stopping, reduce_lr])

# Print the best parameters and score
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')


Best: 0.7237716317176819 using {'epochs': 100, 'batch_size': 64}


In [ ]:
# Create the final model using the best parameters
final_model = create_model(learning_rate=best_params['learning_rate'])
 
# Fit the final model using the entire dataset and the best batch_size and epochs
final_model.fit(X_scaled, y, batch_size=best_params['batch_size'], epochs=best_params['epochs'])

In [166]:
model = Sequential()
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X.shape[1],)))
model.add(BatchNormalization())  # Add Batch Normalization
model.add(Dropout(0.3))  # Add Dropout to prevent overfitting
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1))  # Output layer for regression

In [168]:
optimizer = optimizers.Adam(learning_rate=0.01)  # Use Adam optimizer with learning rate as argument
model.compile(optimizer=optimizer, loss='mae')  # Compile with Mean Absolute Error

In [192]:
model.fit(X, y,  epochs = 100, batch_size = 64)

KerasRegressor(
	model=None
	build_fn=<function create_model at 0x0000029B8CC5C040>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
)

In [194]:
# Evaluate the model on the validation set
val_loss, val_mae = model.evaluate(X, y)
print(f'Validation Mean Absolute Error: {val_mae:.2f}')

AttributeError: 'KerasRegressor' object has no attribute 'evaluate'

In [196]:
# Make predictions on the test set
y_predict = model.predict(X_scaled)

In [202]:
# Create a DataFrame for the predictions
df_predictions = pd.DataFrame(y_predict, columns=['price'])
# Display predictions
print(df_predictions)
combined_df = pd.concat([test_df['Id'], df_predictions], axis=1)
combined_df
combined_df.to_csv('fan.csv', index=False)

            price
0    19151.769531
1    17361.167969
2    22399.242188
3    15271.658203
4     4928.083496
..            ...
406  24225.720703
407  14024.343750
408  13282.091797
409  17809.476562
410  13729.199219

[411 rows x 1 columns]
